<a href="https://colab.research.google.com/github/monjjjjj/orchid_classification/blob/main/orchid_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python import keras
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D
from keras.applications.resnet import ResNet50
from sklearn.model_selection import train_test_split

In [ ]:
# google drive mounted
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read data
labels = []
images = []
paths = []

train_dir = '/content/drive/MyDrive/orchid_classification'
df = pd.read_csv("label.csv")

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)



Found 2191 files belonging to 1 classes.


In [ ]:
def read_image(paths):
  for path in paths:
    image = cv2.imread(path, cv2.COLOR_GRAY2BGR)
    image = image.astype("float") / 255.0
    image = cv2.resize(image,(224,224))
    images.append(image)

for num in range(len(df)):
    paths.append(os.path.join(train_dir, df['filename'][:len(df)][num]))
    labels.append(df['category'][:len(df)][num])

read_image(paths)
images = np.asarray(images)
images = images.reshape((-1, 224, 224, 3))
labels = to_categorical(labels, 219)
print(1, labels)

In [ ]:
# build model
def build_model():
  model = Sequential()